In [ ]:
from random import randint, choice
import tweepy, time, webbrowser, re

class MarkovBot:
    def __init__(self, path_to_secret, path_to_corpus, tweets_a_day): # path to corpus
        self.sleep_timer = int(60 * 60 * tweets_a_day)
        self.transition = {}
        self.start_words = []
        self.api = tweepy.API()
        self.oauth(path_to_secret)
        self.init_corpus(path_to_corpus)
        
    def oauth(self, path_to_secret):
        with open(path_to_secret) as f:
            consumer_key_secret = f.readlines()
        consumer_key_secret = [x.strip() for x in consumer_key_secret] 
        consumer_key = consumer_key_secret[0]
        consumer_secret = consumer_key_secret[1]
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth_url = auth.get_authorization_url()
        webbrowser.open(auth_url)
        verifier = raw_input('PIN: ').strip()
        auth.get_access_token(verifier)
        with open('./access_key_secret.txt', 'w') as f:
            f.write(auth.access_token + '\n' + auth.access_token_secret)
        self.api = tweepy.API(auth)
        
    def init_corpus(self, path_to_corpus):
        with open(path_to_corpus, 'r') as f:
            for line in f.readlines():
                words = line.split()
                self.start_words.append(words[0])
                n = len(words)
                for word1, word2 in zip(words[:(n - 1)], words[1:]):
                    if word1 in self.transition:
                        self.transition[word1].append(word2)
                    else:
                        self.transition[word1] = [word2]
                        
    def generate(self):
        current_word = choice(self.start_words)
        result = [current_word]
        while current_word in self.transition.keys() and self.transition[current_word]:
            current_word = choice(self.transition[current_word])
            result.append(current_word)
        return ' '.join(result)
    
    def get_tweet(self):
        tweet = self.generate()
        while len(tweet.decode('utf8')) > 140 or len(tweet.split()) < 7:
            tweet = self.generate()
        return tweet
        
    def run(self):
        while True:
            tweet = self.get_tweet()
            self.api.update_status(tweet) # Posts to twitter
            time.sleep(self.sleep_timer)

In [ ]:
if __name__ == '__main__':
    bot = MarkovBot('./consumer_key_secret.txt', './cyberleninka_all.txt', 3)
    bot.run()

In [7]:
import tweepy

with open('./consumer.txt') as f:
    consumer = f.readlines()

consumer_key = consumer[0].strip()
consumer_secret = consumer[1]

with open('./access.txt') as f:
    access = f.readlines()

access_token = access[0].strip()
access_token_secret = access[1]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

RT @otmusorov: Когда накуренный убегаешь от мусоров https://t.co/HH7SmfOyXO
RT @cat_wolgast: ебанулся и скачу
Наблюдаю за спорами взрослых дяденек о том как правильно решать проблему слишком открытых прав на ssh ключ, если си… https://t.co/8Ugm2gMiej
RT @AntonBelyayev1: эту запись видят только сладкие булочки, которым сегодня повезёт.
RT @omeliyakitty44: Когда пациент на вызове приносит тебе все снимки и думает, что скоровик в этом разбирается. https://t.co/2kThyIT9O4
RT @TrapOfNerves: господи я так отчаянно пытаюсь донести это до каждой девы 😭😭😭😭 https://t.co/rDa717MKig
Комбо брейкер https://t.co/i8uB59WThl
RT @RussianMemesLtd: If you have at least one saucepan with a cork inserted in the handle, you keep the traditions of your ancestors and pa…
Сегодня планировала ВКАТИТЬСЯ и взяла с собой стрипы в качалку, но там зал для групповых занятий с зеркалами на ремонте.............
RT @ARTSHL3Y: why did i do this https://t.co/4boDQvsNVm
RT @DogsofTikTok: Kittys dreams came true 🥺😍 https://t